In [2]:
from pyscf import gto, scf,ao2mo, fci, mcscf
import numpy as np

mol = gto.Mole()
mol.atom = 'H 0 0 0; H 0 0 0.74'
mol.basis = 'sto-3g'
mol.build()
mf = scf.RHF(mol).run()
norb = mf.mo_coeff.shape[1]
nelec = mol.nelec
h1 = mf.mo_coeff.T @ mf.get_hcore() @ mf.mo_coeff
eri = ao2mo.kernel(mol, mf.mo_coeff)

cisolver = fci.direct_spin1.FCISolver(mol)

# 2. 전체 determinant 개수
ndet = cisolver.dim(norb, nelec)

# 3. 전체 Hamiltonian matrix 생성
H_matrix = np.zeros((ndet, ndet))

# 4. unit vector 만들기
unit_vec = np.eye(ndet)

# 5. 각 basis에 대해 H|Phi_J> 계산 → 내적해서 ⟨Phi_I|H|Phi_J⟩ 얻기
for j in range(ndet):
    H_col = cisolver.contract_2e(eri, unit_vec[j], norb, nelec) + \
            cisolver.contract_1e(h1, unit_vec[j], norb, nelec)
    H_matrix[:, j] = H_col

# 6. 결과 출력 (Slater determinant basis의 full Hamiltonian)
print("Hamiltonian matrix (Slater determinant basis):")
print(H_matrix)


converged SCF energy = -1.11675930739643


AttributeError: 'FCISolver' object has no attribute 'dim'

In [9]:
strings_a = cistring.make_strings(range(norb), nelec[0])
strings_b = cistring.make_strings(range(norb), nelec[1])

print(strings_a)
print(strings_b)

[1 2]
[1 2]


In [11]:
import numpy as np
from pyscf import gto, scf, ao2mo, fci
from pyscf.fci import cistring

# 1. H2 Molecule
mol = gto.Mole()
mol.atom = 'H 0 0 0; H 0 0 0.74'
mol.basis = 'sto-3g'
mol.build()

# 2. SCF
mf = scf.RHF(mol).run()
mo = mf.mo_coeff
norb = mo.shape[1]
# 3. Get integrals
h1 = mo.T @ mf.get_hcore() @ mo
eri = ao2mo.kernel(mol, mo, compact=False).reshape(norb, norb, norb, norb)

# 4. FCI solver
solver = fci.direct_spin1.FCISolver(mol)

# 5. 전체 determinant 개수
na = len(cistring.make_strings(range(norb), nelec[0]))
nb = len(cistring.make_strings(range(norb), nelec[1]))
ndet = na * nb


print(na)
print(nb)

converged SCF energy = -1.11675930739643
2
2


In [ ]:
from pyscf.fci import cistring




def occ_bitstr_to_index(bitstr_alpha, bitstr_beta, norb):
    
    
    """
    PySCF determinant index를 계산하는 함수.
    
    Parameters:
        bitstr_alpha (str): alpha spin orbitals의 occupation (예: '1010')
        bitstr_beta  (str): beta  spin orbitals의 occupation (예: '1100')
        norb (int): 총 spin orbital 개수
    
    Returns:
        int: PySCF determinant index (global index in FCI vector)
    """
    neleca = bitstr_alpha.count("1")
    nelecb = bitstr_beta.count("1")

    strings_a = list(cistring.make_strings(range(norb), neleca))
    strings_b = list(cistring.make_strings(range(norb), nelecb))

    int_a = int(bitstr_alpha, 2)
    int_b = int(bitstr_beta, 2)

    try:
        idx_a = strings_a.index(int_a)
        idx_b = strings_b.index(int_b)
    except ValueError:
        raise ValueError("Bitstring is not a valid determinant in given (norb, nelec) space.")

    return idx_a * len(strings_b) + idx_b

In [34]:
from pyscf.fci import cistring
def occ_bitstr_to_index(bitstr,n):
    norb = 2*n
    bitstr_alpha = bitstr[:norb//2]  # 앞 절반
    bitstr_beta  = bitstr[norb//2:]  # 뒤 절반
    """
    PySCF determinant index를 계산하는 함수.
    
    Parameters:
        bitstr_alpha (str): alpha spin orbitals의 occupation (예: '1010')
        bitstr_beta  (str): beta  spin orbitals의 occupation (예: '1100')
        norb (int): 총 spin orbital 개수
    
    Returns:
        int: PySCF determinant index (global index in FCI vector)
    """
    neleca = bitstr_alpha.count("1")
    nelecb = bitstr_beta.count("1")

    strings_a = list(cistring.make_strings(range(norb), neleca))
    strings_b = list(cistring.make_strings(range(norb), nelecb))

    int_a = int(bitstr_alpha, 2)
    int_b = int(bitstr_beta, 2)

    try:
        idx_a = strings_a.index(int_a)
        idx_b = strings_b.index(int_b)
    except ValueError:
        raise ValueError("Bitstring is not a valid determinant in given (norb, nelec) space.")

    return idx_a * len(strings_b) + idx_b

In [40]:
bit_list = ["0101", "0110", "1001", "1010"]
idx_list = []
for i in bit_list:
    idx = occ_bitstr_to_index(i, 2)
    idx_list.append(idx)

print(idx_list)

[0, 1, 4, 5]


In [42]:
import numpy as np
from pyscf import gto, scf, ao2mo, fci
from pyscf.fci import cistring

import numpy as np
from math import comb


# 1. H2 Molecule
mol = gto.Mole()
mol.atom = 'H 0 0 0; H 0 0 0.74'
mol.basis = 'sto-3g'
mol.build()

# 2. SCF
mf = scf.RHF(mol).run()
mo = mf.mo_coeff
norb = mo.shape[1]
nelec = mol.nelec  # (1,1) for H2

# 3. Get integrals
h1 = mo.T @ mf.get_hcore() @ mo
eri = ao2mo.kernel(mol, mo, compact=False).reshape(norb, norb, norb, norb)

# 4. FCI solver
solver = fci.direct_spin1.FCISolver(mol)
# 5. 전체 determinant 개수
na = comb(norb, nelec[0])  # alpha
nb = comb(norb, nelec[1])  # beta
ndet = na * nb

print(ndet)


# 6. 단위 벡터 (전체 공간에서)
unit_vec = np.eye(ndet)

bit_list = ["0101", "0110", "1001", "1010"]
idx_list = []
for i in bit_list:
    idx = occ_bitstr_to_index(i, 2)
    idx_list.append(idx)
# 7. 내가 선택한 determinant index 리스트
selected_dets = idx_list   # 예시
k = len(selected_dets)

# 8. projection된 subspace Hamiltonian 만들기
H_sub = np.zeros((k, k))

for j_idx, j in enumerate(selected_dets):
    print
    Hj = solver.contract_1e(h1, unit_vec[j], norb, nelec) + \
         solver.contract_2e(eri, unit_vec[j], norb, nelec)
    print(Hj)
    for i_idx, i in enumerate(selected_dets):
        H_sub[i_idx, j_idx] = Hj[i]

# 9. 결과 출력
print("Projected Hamiltonian in selected determinant subspace:")
print(H_sub)

converged SCF energy = -1.11675930739643
4
[5.54825058e-01 3.85458074e-17 3.85458074e-17 3.62420924e-01]


IndexError: index 4 is out of bounds for axis 0 with size 4